# Advanced

In this section we will consider some advanced aspects related to the package.

## Performance considerations

### Solving the dual versus the primal formulation of the SDP

For semidefinite programs that appear often in causal compatibility problems, using the dual formulation speeds up the solve time and significantly lowers RAM usage.

Consider the following example, where we use the MOSEK Fusion API to solve the primal version of a program, and then the dual:

In [1]:
from inflation import InflationProblem, InflationSDP
from time import time
import numpy as np

qtriangle = InflationProblem(dag={"rho_AB": ["A", "B"],
                                  "rho_BC": ["B", "C"],
                                  "rho_AC": ["A", "C"]}, 
                             outcomes_per_party=[2, 2, 2],
                             settings_per_party=[1, 1, 1],
                             inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle, verbose=0)
sdprelax.generate_relaxation('npa2')

P_W = np.zeros((2,2,2,1,1,1))
for a, b, c in np.ndindex((2,2,2)):
    if a + b + c == 1:
        P_W[a,b,c,0,0,0] = 1/3

sdprelax.set_distribution(P_W)

time0 = time()
sdprelax.solve(solve_dual=False)
print("The primal formulation was solved in", time()-time0, "seconds.")
time0 = time()
sdprelax.solve(solve_dual=True)
print("The dual formulation was solved in", time()-time0, "seconds.")

The primal formulation was solved in 20.820358276367188 seconds.
The dual formulation was solved in 0.8410844802856445 seconds.


Notice that there is an order of magnitude difference between the primal and dual formulations of the same problem. This is not true for all problems, but for the semidefinite programming relaxations generated for causal compatibility, almost always the dual formulation is more efficient. This should be taken into account when attempting to solve a relaxation. In what follows, we recompile some useful information for different interfaces.

- [CVXPY](https://www.cvxpy.org/). If you export the problem to CVXPY, the behaviour depends on the solver you choose to use. When choosing MOSEK, note that CVXPY [dualises by default](https://www.cvxpy.org/tutorial/advanced/index.html?highlight=dualization) all continuous problems. There is [no automatic dualisation option](https://github.com/cvxpy/cvxpy/issues/1403). There is no option to specify whether to solve the primal or dual problem. Thus if you wanted to solve the primal with MOSEK, you would need to write the dual formulation manually, which when dualised would solve the primal (it is not expected that the user will need to do this!).
- [PICOS 2.4](https://picos-api.gitlab.io/picos/). Picos [supports dualisation](https://picos-api.gitlab.io/picos/api/picos.modeling.options.html#option-dualize) with the `dualise=True` options flag. See [this issue](https://gitlab.com/picos-api/picos/-/issues/280) for more details. 
- [YALMIP](https://yalmip.github.io/). Like CVXPY, YALMIP [automatically dualises](https://yalmip.github.io/tutorial/automaticdualization) problems, however there is a flag, `dualize`, in `sdpsettings` to disable this feature if so desired.
- MOSEK Fusion API. Our implementation of the semidefinite programming relaxation supports both the primal and dual formulations, as seen in the example above. This is done manually, as MOSEK Fusion API does not have functionality to change from the primal to the dual formulations.


### Large scale problems

For solving large scale semidefinite programs, it is recommended to use the MOSEK Fusion API, as going through interfaces for conic problems, such as PICOS or CVXPY, usually has an overhead in the pre-processing state (for example, there can be a higher RAM usage in the preprocessing stage than when solving the problem, which can lead to out-of-memory errors). There does not seem to be such an overhead when using YALMIP. For using YALMIP, the user can export the problem to `.dat-s` format using `InflationSDP.write_to_file()`, and load it in MATLAB using YALMIP's `loadsdpafile`.

For large problems, it is recommended to try using a first-order SDP solver, such as [SCS](https://www.cvxgrp.org/scs/), if using second-order SDP solvers, such as MOSEK, is too slow or too memory-consuming. To use SCS the problem needs to be exported to the user's interface of choice and have SCS installed.

It is also worth considering using symmetries to block-diagonalise the semidefinite program. This can be done with [RepLAB](https://replab.github.io/web/) in MATLAB. Symmetries arising from inflation are stored in `InflationSDP.columns_symmetries`, and they are encoded as permutations of the list of generating monomials which leave the SDP invariant. This then can be used in RepLAB to block-diagonalise the problem (see this [example from RepLAB](https://replab.github.io/applis/SDP.html#Block-diagonalizing-a-symmetric-SDP-matrix)). A more in-depth example with code detailing this will be added to the Examples section in the future.


### Optimization under distributions with symmetries

In some cases, we know that the optimum of the problem at hand has certain symmetries, such as invariance under permutation of parties or under relabelling of outputs. In such cases, restricting the optimization to the symmetric subspace leads to memory and time savings. Importantly, inserting these symmetries does not alter the feasibility of the problem if we know that the optimal satisfies the symmetries. Otherwise, the optimization is performed only under the set of distributions that satisfy the symmetries.

Symmetries can be inserted into an `InflationProblem` via the method `add_symmetries()`. Once this is done, the subsequent `InflationLP` or `InflationSDP` takes into account these symmetries when generating the optimization problems. Symmetries are stored as permutations of the lexicographical order of the building blocks of the atomic monomials. This lexicographical ordering is stored in `InflationProblem._lexorder`. The `symmetry_utils` submodule contains useful functions to generate the symmetries for a given scenario. In particular, it contains the function `discover_distribution_symmetries()`, that automatically finds the symmetries of a given distribution that are compatible with a given inflation problem.

Below we demonstrate the gains on a simple problem in the triangle network.

In [ ]:
import numpy as np
from itertools import permutations

from inflation import InflationProblem, InflationLP
from inflation.symmetry_utils import discover_distribution_symmetries

d = 3
triangle = InflationProblem(dag={"AB": ["A", "B"],
                                 "BC": ["B", "C"],
                                 "AC": ["A", "C"]},
                            outcomes_per_party=(d,d,d),
                            inflation_level_per_source=(2,2,2),
                            order=["A","B","C"],
                            classical_sources='all')

# Distribution taken from https://github.com/seemann5/recursive-sudoku
p = np.zeros(shape=(d,d,d,1,1,1), dtype=float)
for a in range(d):
    for b in range(0,a):
        for c in range(0,b):
            val = (1-1/d)/(d*(d-1)*(d-2))
            indices_choices = permutations((a,b,c))
            for indices in indices_choices:
                p[indices] = val
    p[a,a,a] = 1/d*1/d

triangleLP = InflationLP(triangle)
triangleLP.set_distribution(p, use_lpi_constraints=True)
print(f"Un-symmetrized: Known variables: {triangleLP.n_knowable}, "
      + f"semi-known variables: {triangleLP.n_something_knowable}, "
      + f"unknown variables: {triangleLP.n_unknowable}")

p_symmetries = discover_distribution_symmetries(p, triangle)
triangle.add_symmetries(p_symmetries)

triangleLP = InflationLP(triangle)
triangleLP.set_distribution(p, use_lpi_constraints=True)
print(f"Symmetrized: Known variables: {triangleLP.n_knowable}, "
      + f"semi-known variables: {triangleLP.n_something_knowable}, "
      + f"unknown variables: {triangleLP.n_unknowable}")


Un-symmetrized: Known variables: 386, semi-known variables: 624, unknown variables: 68245
Symmetrized: Known variables: 16, semi-known variables: 23, unknown variables: 5478


In this case, we obtain a reduction in variables of a factor of ~10. The memory savings are such that they determine the problem being runnable on a standard laptop or needing >50 GB of RAM for doing so.